In [23]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import math
from jupyterthemes import jtplot
jtplot.style(theme='monokai')
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')
jtplot.style(ticks=True, grid=False, figsize=(10, 10))
import datetime
from psaw import PushshiftAPI


In [24]:
api = PushshiftAPI()
features = ["title", "id", "score", "author", "created_utc", "selftext"]
subreddit = "asoiaf"
date1 = int(datetime.datetime(2020,1,1).timestamp())
date2 = int(datetime.datetime(2019,4,14).timestamp())
gen = api.search_submissions(subreddit = subreddit, after = date1, before = date2, q='', filter = features, limit = 10)

In [25]:
results = list(gen)

In [27]:

ids, title, score, author, created_utc, selftext = [], [], [], [], [], []
for i in range(len(results)):
    ids.append(i)
    title.append(results[i].d_['title'])
    score.append(results[i].d_['score'])
    created_utc.append(results[i].d_['created_utc'])
    author.append(results[i].d_['author'])
    selftext.append(results[i].d_['selftext'])

data = pd.DataFrame(
{
    "id" : ids,
    "title" : title,
    "score" : score,
    "created" : created_utc,
    "author" : author,
    "selftext" : selftext
}, index = None)

In [28]:
data

,id,title,score,created,author,selftext
0,0,If someone were to watch the first 4 seasons o...,1,1611529121,popejuliusii,I have a friend who wants to get into the univ...
1,1,Kintsugi and the Ephemeral Beauty of Broken Me...,1,1611527682,JT_Soul,"""The wound is the place where the Light enters..."
2,2,[Spoilers Main] Is it possible that we have Fa...,1,1611527395,jzimoneaux,"After watching some of InDeepGeek’s videos, he..."
3,3,(Spoilers Main) Who hates and loves grrm world...,1,1611525863,unknown2574,I mean we are never get answers to more them h...
4,4,(Spoilers Extended) So the Drama (Varys: Early...,1,1611524125,kaimkre1,"Evocative images, strong, sparse descriptors, ..."
5,5,(Spoilers Extended) Volantis was involved in c...,1,1611523847,GenghisKazoo,**TL;DR Volantis is sus**\n\nConsider this a f...
6,6,Officers of the Nights Watch,1,1611523560,JcapitalcityDC,[removed]
7,7,House Martell [No Spoilers],1,1611523354,kryptonzera,[removed]
8,8,House Martell,1,1611523131,kryptonzera,[removed]
9,9,Who started reading the books after watching t...,1,1611522656,lewiss15,[removed]\n\n[View Poll](https://www.reddit.co...
